In [ ]:
CREATE OR REPLACE TEMP VIEW vw_cost_center_mapping_bootstrap AS

WITH Primary_AU AS (
    -- 1. Grab the clean, primary AUs directly from the base columns
    SELECT 
        `CostCenter` AS Cost_Center_ID,
        TRIM(CAST(`AssessableUnitID` AS STRING)) AS AU_ID,
        TRIM(`AssessableUnitName`) AS AU_Name,
        TRIM(`Segment`) AS Segment_Name
    FROM fy25_cost_center_mapping
    WHERE `AssessableUnitID` IS NOT NULL
),

Additional_Strings AS (
    -- 2. Handle the Col E "Yes" rule to build our string for parsing
    SELECT 
        `CostCenter`,
        CASE 
            WHEN TRIM(`AdditionalAssessableUnitIDandNameandSegment`) = 'Yes' 
            THEN COALESCE(`AdditionalAUID`, '')
            
            ELSE CONCAT_WS(' ', 
                    COALESCE(`AdditionalAssessableUnitIDandNameandSegment`, ''), 
                    COALESCE(`AdditionalAUID`, '')
                 )
        END AS Mashed_String
    FROM fy25_cost_center_mapping
),

Extracted_Blocks AS (
    -- 3. Slice the mashed text into discrete [ID - Name - Segment] string blocks
    SELECT 
        `CostCenter`,
        EXPLODE(regexp_extract_all(Mashed_String, '(\\d{6}.*?(?=\\d{6}|$))')) AS Raw_Block
    FROM Additional_Strings
    WHERE Mashed_String != ''
),

Parsed_Additionals AS (
    -- 4. Parse the discrete strings into your requested columns
    SELECT 
        `CostCenter` AS Cost_Center_ID,
        
        -- Grabs the first 6 digits
        regexp_extract(TRIM(Raw_Block), '^(\\d{6})', 1) AS AU_ID,
        
        -- Smart extraction: Grabs everything between the first hyphen and the LAST hyphen. 
        -- (This safely protects any internal hyphens inside the AU_Name!)
        TRIM(regexp_extract(TRIM(Raw_Block), '^\\d{6}\\s*-\\s*(.*)\\s*-\\s*[^-]+$', 1)) AS AU_Name,
        
        -- Grabs everything strictly after the last hyphen
        TRIM(regexp_extract(TRIM(Raw_Block), '.*\\s*-\\s*([^-]+)$', 1)) AS Segment_Name
        
    FROM Extracted_Blocks
    WHERE TRIM(Raw_Block) != ''
)

-- 5. Stack the Primary AUs and the Parsed Additional AUs together.
-- Standard UNION natively forces the final output to be strictly DISTINCT.
SELECT * FROM Primary_AU
UNION
SELECT * FROM Parsed_Additionals
WHERE AU_ID != '' -- Drops any failed parses